<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/TestingModelos/Models/YoloV8Small_100Epochs/TrackingBotsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generando Video de Tracking con modelo Small

In [ ]:
%%capture
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
%%capture
!wget https://www.dropbox.com/scl/fi/g1ozmpe6y9k48cxc041rp/bestSmall100.pt?rlkey=fs2aktr3tdkay39lg3gw9x9tb&st=iynm04em&dl=0
!mv bestSmall100.pt?rlkey=fs2aktr3tdkay39lg3gw9x9tb bestSmall100.pt

--2024-05-11 17:16:16--  https://www.dropbox.com/scl/fi/g1ozmpe6y9k48cxc041rp/bestSmall100.pt?rlkey=fs2aktr3tdkay39lg3gw9x9tb
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc50195311bd305c5dd640953d37.dl.dropboxusercontent.com/cd/0/inline/CSuxErDlMPoC3bXvypy4sWWTiqD6OZw1cQ2IBdGn9qKJpcgGXcsBoeNRHDsC_8nttnImFzO7lRTRotbzuJHh9rgYSK-lHdWW1YkTgJaAQ2ig0shDifHe60fyCPFsqM4wwmcCEPT4yyk5tEhweRgvhJvG/file# [following]
--2024-05-11 17:16:17--  https://uc50195311bd305c5dd640953d37.dl.dropboxusercontent.com/cd/0/inline/CSuxErDlMPoC3bXvypy4sWWTiqD6OZw1cQ2IBdGn9qKJpcgGXcsBoeNRHDsC_8nttnImFzO7lRTRotbzuJHh9rgYSK-lHdWW1YkTgJaAQ2ig0shDifHe60fyCPFsqM4wwmcCEPT4yyk5tEhweRgvhJvG/file
Resolving uc50195311bd305c5dd640953d37.dl.dropboxusercontent.com (uc50195311bd305c5dd640953d37.dl.dropboxusercontent.com)... 162.125.5.15, 2

In [ ]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO('bestSmall100.pt')

In [ ]:
%%capture
!wget https://www.dropbox.com/scl/fi/v0t8noyhzp1eutaj13w6y/VideosTrackingTest.zip?rlkey=pxdttv3eepxfvglud37qkzegr&st=jw9bqwug&dl=0
!mv VideosTrackingTest.zip?rlkey=pxdttv3eepxfvglud37qkzegr VideosTrackingTest.zip

--2024-05-11 17:18:04--  https://www.dropbox.com/scl/fi/v0t8noyhzp1eutaj13w6y/VideosTrackingTest.zip?rlkey=pxdttv3eepxfvglud37qkzegr
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc17cfa4588dad79843f98761cc.dl.dropboxusercontent.com/cd/0/inline/CSt_SgRTosqYixclG4mRR-36hii2WMrU453XuvRKxOMEn8RT47p3V0yf72NE7RhRkKHJR_Txk2tdNsnwTJ73CoGEFunoZl_Z4rQPZkW6x8OJkwlAMDqAsFI6YazC-f2QKga7cP6zCmS3BAM_EBaLrPZV/file# [following]
--2024-05-11 17:18:05--  https://ucc17cfa4588dad79843f98761cc.dl.dropboxusercontent.com/cd/0/inline/CSt_SgRTosqYixclG4mRR-36hii2WMrU453XuvRKxOMEn8RT47p3V0yf72NE7RhRkKHJR_Txk2tdNsnwTJ73CoGEFunoZl_Z4rQPZkW6x8OJkwlAMDqAsFI6YazC-f2QKga7cP6zCmS3BAM_EBaLrPZV/file
Resolving ucc17cfa4588dad79843f98761cc.dl.dropboxusercontent.com (ucc17cfa4588dad79843f98761cc.dl.dropboxusercontent.com)... 162.125.5.

In [ ]:
%%capture
!unzip VideosTrackingTest.zip

Archive:  VideosTrackingTest.zip
  inflating: VideosTrackingTest/acA1920-155um__22949301__20200710_094124535.avi  
  inflating: VideosTrackingTest/acA1920-155um__22949301__20210625_105958792.avi  
  inflating: VideosTrackingTest/acA1920-155um__22949301__20210628_094625267.avi  
  inflating: VideosTrackingTest/acA1920-155um__22949301__20210628_114549599.avi  
  inflating: VideosTrackingTest/acA1920-155um__22949301__20220526_124553810.avi  
  inflating: VideosTrackingTest/acA1920-155um__22949301__20220526_160756867.avi  
  inflating: VideosTrackingTest/Suppl_1.avi  


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from collections import defaultdict
import os

cv2.imshow = cv2_imshow


In [ ]:
# Directorio de entrada y salida
input_folder = "VideosTrackingTest/"
output_folder = "output_videos/"

# Crear el directorio de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Listar los archivos de video en el directorio de entrada
video_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Iterar sobre cada archivo de video
for video_file in video_files:
    # Abrir el archivo de video de entrada
    video_path = os.path.join(input_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    # Verificar si la apertura del video fue exitosa
    if not cap.isOpened():
        print(f"Error al abrir el archivo de video: {video_file}")
        continue

    # Obtener información del video de entrada
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (frame_width, frame_height)

    # Crear el objeto VideoWriter para el video de salida
    output_video_file = os.path.join(output_folder, f"{os.path.splitext(video_file)[0]}_Out.avi")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Formato de compresión de vídeo
    out = cv2.VideoWriter(output_video_file, fourcc, fps, frame_size)

    # Store the track history
    track_history = defaultdict(lambda: [])

    model = YOLO('bestSmall100.pt')

    # Loop through the video frames
    while cap.isOpened():
        # Leer un cuadro del video
        success, frame = cap.read()

        if success:
            # Ejecutar el seguimiento YOLOv8 en el cuadro, persistiendo los tracks entre cuadros
            results = model.track(frame, persist=True)

            # Obtener las cajas y los IDs de seguimiento
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualizar los resultados en el cuadro
            annotated_frame = results[0].plot()

            # Dibujar las líneas de seguimiento
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y centro
                if len(track) > 80:  # mantiene 80 tracks por 80 cuadros
                    track.pop(0)

                # Dibujar las líneas de seguimiento
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=3)

            # Escribir el cuadro procesado en el archivo de salida
            out.write(annotated_frame)

        else:
            # Terminar la ejecución si se alcanza el final del video
            break

    # Liberar los recursos
    cap.release()
    out.release()


0: 768x1216 31 Movils, 1491.2ms
Speed: 11.1ms preprocess, 1491.2ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 1821.4ms
Speed: 12.2ms preprocess, 1821.4ms inference, 2.0ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 2286.5ms
Speed: 16.2ms preprocess, 2286.5ms inference, 1.9ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 1503.0ms
Speed: 22.4ms preprocess, 1503.0ms inference, 1.9ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 1477.6ms
Speed: 13.0ms preprocess, 1477.6ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 1473.2ms
Speed: 12.4ms preprocess, 1473.2ms inference, 1.3ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 1446.6ms
Speed: 17.9ms preprocess, 1446.6ms inference, 1.3ms postprocess per image at shape (1, 3, 768, 1216)

0: 768x1216 31 Movils, 1438.3ms
Speed: 21.5ms preprocess, 143

In [ ]:
import shutil
archivo = shutil.make_archive('VideosOutTrackingSmall', 'zip','/content/output_videos')